# Demo 
___

### 1. Chargement des données 

* UCR : https://www.cs.ucr.edu/~eamonn/time_series_data_2018/

* UEA : http://www.timeseriesclassification.com/

* Consommation éléctrique (puissance énergétique) journalière individuelle : https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption

### 2. Modèles + entrainements

### 3. Résultats